In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# tf.RandomShuffleQueue

N_SAMPLES = 1000
NUM_THREADS = 4
N_ITERATIONS = 100

# Generating simple data
data = 10 * np.random.randn(N_SAMPLES, 4) + 1
# random labels
target = np.random.randint(0, 2, size=N_SAMPLES)

queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])
enqueue_op = queue.enqueue_many([data, target])
dequeue_op = queue.dequeue()

# create NUM_THREADS
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in xrange(N_ITERATIONS):
        if coord.should_stop():
            break
        data_batch, label_batch = sess.run(dequeue_op)
    coord.request_stop()
    coord.join(enqueue_threads)